In [0]:
from pandas import read_csv
dataframe = read_csv('Thailand3D1.csv', header=0,engine='python',usecols=range(1,1351))
dataframe.shape

(36404, 71)

In [0]:
from keras.layers import TimeDistributed
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [0]:
no_days=1916
no_data_per_day=19
features=71

In [0]:
dataset = dataframe.values
#print(dataset[4,:])

In [0]:
print(dataframe.shape)
#print(dataframe.iloc[1,:])
dataset = dataframe.values
#print(dataset[2,:])
dataset = dataset.astype('float32')
#print(dataset.shape)
#print(dataset[0])
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
#print(dataset[1,:])
dataset = dataset.reshape(no_days,no_data_per_day,features)# no. of days, days data, features
#print(dataset.shape)
#print(dataset[0,:,:])

(36404, 71)


In [0]:
print(dataset.shape)
#train_size = int(len(dataset) * 0.67)
#test_size = len(dataset) - train_size
train_size=1826
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(train.shape, test.shape)
#print (train)
#print(train[0])
#print(test[0])

(1916, 19, 71)
(1826, 19, 71) (90, 19, 71)


In [0]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = train[i:(i+look_back),:,:]
		dataX.append(a)
		dataY.append(train[i + look_back,:,:])
	return numpy.array(dataX), numpy.array(dataY)

In [0]:
import numpy
# reshape into X=t and Y=t+1
look_back = 1
print("Size of data sent:",train.shape)
print(len(train)-look_back-1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
#print(trainX)
print(trainX.shape)
#print(trainY)
print(trainY.shape)
#print(trainX.reshape(no_days,1,no_data_per_day*features))
print(no_data_per_day, features)

print(no_data_per_day*features)
trainX = trainX.reshape(1824,1,no_data_per_day*features)#9861
trainY = trainY.reshape(1824,no_data_per_day*features)

Size of data sent: (1826, 19, 71)
1824
(1824, 1, 19, 71)
(1824, 19, 71)
19 71
1349


In [0]:
pip install phased_lstm_keras

In [0]:
import numpy as np
print(np.any(np.isnan(trainX)))
print(np.any(np.isnan(trainY)))

False
False


In [0]:
from keras.layers import Masking
from phased_lstm_keras.PhasedLSTM import PhasedLSTM as PLSTM
a=no_data_per_day*features
# create and fit the LSTM network
model = Sequential()
#model.add(Masking(mask_value=3.6122181e+00, batch_input_shape=(None,None, a)))
#(output size,batch_input_shape:input shape of the data)
model.add(PLSTM(a))#, batch_input_shape=(None,None, a),return_sequences=False))
#model.add(TimeDistributed(Dense(5)))
model.add(Dense(a))
#model.add(Dropout(1))
#model.add(Dense(a))
model.compile(loss='mean_squared_error', optimizer='adam')
#print(model.summary())
model.fit(trainX, trainY, epochs=2, batch_size=20, verbose=2)
#model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=0)


Epoch 1/2
 - 4s - loss: 0.1471
Epoch 2/2
 - 2s - loss: 0.0097


In [0]:
model.save('model_thailand.h5')

In [0]:
trainPredict = model.predict(trainX)
print(trainPredict.shape)
X=trainPredict.reshape(1824,no_data_per_day,features)
print(X.shape)
print(type(X))
A=[]
for i in range(len(X-1)):
  a = scaler.inverse_transform(X[i])
  A.append(a)
#print(A)
print(trainY.shape)
Y=trainY.reshape(1824,no_data_per_day,features)
B=[]
for i in range(len(Y-1)):
  b = scaler.inverse_transform(Y[i])
  B.append(b)
print(type(B))
trainY=numpy.asarray(B).reshape(1824,no_data_per_day*features)
trainPredict=numpy.asarray(A).reshape(1824,no_data_per_day*features)
#print(B)
#trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))

(1824, 1349)
(1824, 19, 71)
<class 'numpy.ndarray'>
(1824, 1349)
<class 'list'>


In [0]:
testX = testX.reshape(88,1,no_data_per_day*features)
testPredict = model.predict(testX)
X=testPredict.reshape(88,no_data_per_day,features)
A=[]
for i in range(len(X-1)):
  a = scaler.inverse_transform(X[i])
  print('Append:',a)
  A.append(a)
#print(A)
print('testY',testY.shape)
Y=testY.reshape(88,no_data_per_day,features)
B=[]
for i in range(len(Y-1)):
  b = scaler.inverse_transform(Y[i])
  B.append(b)
print(type(B))
testY=numpy.asarray(B).reshape(88,no_data_per_day*features)
testPredict=numpy.asarray(A).reshape(88,no_data_per_day*features)
#print(B)
#trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))

Append: [[  1.82291     2.0709307   2.3348374 ...   3.6290176  11.333609
  100.761406 ]
 [  2.5501244   2.6444724   2.7790742 ...   3.6328814  11.654343
  100.85163  ]
 [  3.0738416   3.144085    3.176826  ...   3.626531   12.876408
  100.7425   ]
 ...
 [  3.6261775   3.615466    3.6161282 ...   3.6388521  15.687239
  102.1787   ]
 [  3.6127949   3.615478    3.6073422 ...   3.6203372  16.429985
  102.82217  ]
 [  3.6075869   3.6154058   3.6057243 ...   3.62994    16.455603
  102.83192  ]]
Append: [[  1.8229252   2.070972    2.3348963 ...   3.629078   11.33377
  100.761406 ]
 [  2.5501518   2.644541    2.7791073 ...   3.6329029  11.654309
  100.8516   ]
 [  3.073846    3.1440704   3.176795  ...   3.6265805  12.87631
  100.74251  ]
 ...
 [  3.6262546   3.6154776   3.6161225 ...   3.638944   15.687312
  102.17872  ]
 [  3.6128716   3.6155763   3.6073723 ...   3.6203718  16.43017
  102.82214  ]
 [  3.6076453   3.6153905   3.6058176 ...   3.629979   16.45585
  102.83192  ]]
Append: [[  1.82

In [0]:
print(testPredict.shape)
lastPred=testPredict[87:88,:]
lastPredX = numpy.reshape(lastPred, (lastPred.shape[0], 1, 1349))#1700=85*20
predictNext = model.predict(lastPredX)
P=predictNext.reshape(1,no_data_per_day,features)
PN=[]
for i in range(len(P-1)):
  a = scaler.inverse_transform(P[i])
  print('Append:',a)
  A.append(a)
print("Next day prediction Done")

(88, 1349)
Append: [[  1.822905    2.0711796   2.333361  ...   3.6294737  11.332145
  100.75899  ]
 [  2.550444    2.6435142   2.7777312 ...   3.6312847  11.654269
  100.854225 ]
 [  3.0739865   3.1429062   3.1766193 ...   3.6252396  12.874331
  100.73946  ]
 ...
 [  3.6242568   3.6147745   3.614375  ...   3.6384237  15.687484
  102.1763   ]
 [  3.6115403   3.614727    3.6068544 ...   3.6199617  16.427818
  102.82406  ]
 [  3.6056874   3.6146855   3.603278  ...   3.627602   16.45392
  102.832054 ]]
Next day prediction Done


In [0]:
days = input("Number of Days to predict?")
days=int(days)
for i in range(days):
  Next=predictNext
  todayPredictionX = numpy.reshape(Next, (lastPred.shape[0], look_back, 1349))
  predictNext = model.predict(todayPredictionX)
  P=predictNext.reshape(1,no_data_per_day,features)
  for i in range(len(P-1)):
    a = scaler.inverse_transform(P[i])
    print('Append:',a)
    A.append(a)

Number of Days to predict?47
Append: [[  1.822911    2.0709565   2.3348823 ...   3.6290634  11.333793
  100.76139  ]
 [  2.5501306   2.6445217   2.779093  ...   3.6329052  11.654355
  100.85159  ]
 [  3.0738535   3.1440568   3.1768036 ...   3.6265707  12.876354
  100.74249  ]
 ...
 [  3.6262367   3.6154828   3.6161184 ...   3.6389186  15.687325
  102.1787   ]
 [  3.6128545   3.615553    3.6073632 ...   3.62037    16.43008
  102.82215  ]
 [  3.6076326   3.6154025   3.6057978 ...   3.6299741  16.45577
  102.83193  ]]
Append: [[  1.8229108   2.0709562   2.3348825 ...   3.6290634  11.333795
  100.7614   ]
 [  2.5501306   2.6445217   2.779093  ...   3.6329057  11.654355
  100.85159  ]
 [  3.0738535   3.1440573   3.1768036 ...   3.6265707  12.876354
  100.74249  ]
 ...
 [  3.6262367   3.6154828   3.6161184 ...   3.6389186  15.687325
  102.1787   ]
 [  3.6128545   3.615553    3.6073635 ...   3.62037    16.430082
  102.82215  ]
 [  3.6076329   3.6154025   3.6057985 ...   3.629975   16.455769
 

In [0]:
import numpy as np
W = np.array(A)
W.shape

(136, 19, 71)

In [0]:
import pandas as pd
P = W.reshape(W.shape[0],19*features)
pd.DataFrame(P).to_csv('PredictedThai.csv')

# NORMALIZE OUTPUT

In [0]:
from pandas import read_csv
dataframe = read_csv('PredictedThai.csv', header=0,engine='python',usecols=range(1,20000))
dataframe.shape

(136, 1349)

In [0]:
d=dataframe.values

for i in range(d.shape[0]-1):
  if abs(d[i,69]-d[i+1,69])<0.0001:
    d[i,69]=0
    d[i,70]=0
  if abs(d[i,140]-d[i+1,140])<0.0001:
    d[i,140]=0     
    d[i,141]=0
  if abs(d[i,211]-d[i+1,211])<0.0001:
    d[i,211]=0     
    d[i,212]=0
  if abs(d[i,282]-d[i+1,282])<0.0001:
    d[i,282]=0     
    d[i,283]=0
  if abs(d[i,353]-d[i+1,353])<0.0001:
    d[i,353]=0     
    d[i,354]=0
  if abs(d[i,424]-d[i+1,424])<0.0001:
    d[i,424]=0     
    d[i,425]=0
  if abs(d[i,495]-d[i+1,495])<0.0001:
    d[i,495]=0     
    d[i,496]=0
  if abs(d[i,566]-d[i+1,566])<0.0001:
    d[i,566]=0     
    d[i,567]=0
  if abs(d[i,637]-d[i+1,637])<0.0001:
    d[i,637]=0     
    d[i,638]=0
  if abs(d[i,708]-d[i+1,708])<0.0001:
    d[i,708]=0     
    d[i,709]=0
  if abs(d[i,779]-d[i+1,779])<0.0001:
    d[i,779]=0     
    d[i,780]=0
  if abs(d[i,850]-d[i+1,850])<0.0001:
    d[i,850]=0     
    d[i,851]=0
  if abs(d[i,921]-d[i+1,921])<0.0001:
    d[i,921]=0     
    d[i,922]=0
  if abs(d[i,992]-d[i+1,992])<0.0001:
    d[i,992]=0     
    d[i,993]=0
  if abs(d[i,1063]-d[i+1,1063])<0.0001:
    d[i,1063]=0     
    d[i,1064]=0
  if abs(d[i,1134]-d[i+1,1134])<0.0001:
    d[i,1134]=0     
    d[i,1135]=0
  if abs(d[i,1205]-d[i+1,1205])<0.0001:
    d[i,1205]=0     
    d[i,1206]=0
  if abs(d[i,1276]-d[i+1,1276])<0.0001:
    d[i,1276]=0     
    d[i,1277]=0
  if abs(d[i,1347]-d[i+1,1347])<0.0001:
    d[i,1347]=0     
    d[i,1348]=0
  

In [0]:
import pandas as pd

pd.DataFrame(d).to_csv('Pred1Thai.csv')

In [0]:
print(pd.DataFrame(d).shape)
df = pd.DataFrame(d)
d = d.reshape(d.shape[0],19,71)

(136, 1349)


In [0]:
pip install reverse_geocoder

In [0]:
print(d.shape)

W=d[124:136,:]
print(W.shape)
W=d

(136, 19, 71)
(12, 19, 71)


In [0]:
import datetime
import pandas as pd
import reverse_geocoder as rg 
mydate = datetime.date(2019,1,1)  #year, month, day
i=0
row_list=[]

for i in range(W.shape[0]):
  dt = datetime.datetime.strptime(str(mydate), '%Y-%m-%d')
  date='{0}/{1}/{2:02}'.format(dt.month%100, dt.day%100, dt.year)
  print(date)
  for j in range(19):
    if  W[i,j,69]==0:
      continue
    revGeo=dict.fromkeys(['date','lat','lon','state','country','place'])
   
    
    #revGeo['lat']= W[i,j,69]
    #revGeo['lon']=W[i,j,70]
    #print(W[i,j,69],W[i,j,70])
    coordinates = (W[i,j,69],W[i,j,70])
    print(coordinates)
    result = rg.search(coordinates)
    #print(result[0])
    print(result[0]["cc"])
    if(result[0]['cc']=='TH'):
      #print(result[0]['cc'])
      revGeo['date']=date
      revGeo['lat']=result[0]['lat']
      revGeo['lon']=result[0]['lon']
      revGeo['state']=result[0]['admin1']
      revGeo['country']=result[0]['cc']
      revGeo['place']=result[0]['name']
      row_list.append(revGeo)
  mydate=mydate+datetime.timedelta(days=1)
  #print()
  #print()


1/1/2019
1/2/2019
1/3/2019
(16.45585, 102.83192)
TH
1/4/2019
1/5/2019
1/6/2019
1/7/2019
1/8/2019
1/9/2019
1/10/2019
1/11/2019
1/12/2019
(13.573891, 100.24852)
TH
(14.113544000000001, 100.35831999999999)
TH
(15.737314000000001, 102.20013)
TH
(16.455791, 102.8319)
TH
1/13/2019
(13.573473000000002, 100.24848)
TH
(14.113218, 100.35847)
TH
(16.45533, 102.83175)
TH
1/14/2019
1/15/2019
1/16/2019
1/17/2019
1/18/2019
1/19/2019
1/20/2019
1/21/2019
1/22/2019
1/23/2019
1/24/2019
1/25/2019
1/26/2019
1/27/2019
1/28/2019
1/29/2019
1/30/2019
1/31/2019
2/1/2019
2/2/2019
2/3/2019
2/4/2019
2/5/2019
2/6/2019
2/7/2019
2/8/2019
2/9/2019
2/10/2019
2/11/2019
2/12/2019
2/13/2019
2/14/2019
2/15/2019
2/16/2019
2/17/2019
2/18/2019
2/19/2019
2/20/2019
2/21/2019
2/22/2019
2/23/2019
2/24/2019
2/25/2019
2/26/2019
2/27/2019
2/28/2019
3/1/2019
3/2/2019
3/3/2019
3/4/2019
3/5/2019
3/6/2019
3/7/2019
3/8/2019
3/9/2019
3/10/2019
3/11/2019
3/12/2019
3/13/2019
3/14/2019
3/15/2019
3/16/2019
3/17/2019
3/18/2019
3/19/2019
3/20/2

In [0]:
pd.DataFrame(row_list).to_csv('ThailandPrediction2019Future.csv')

In [0]:
with open('ThailandPrediction2019Future.txt', 'w') as f:
    for item in row_list:
        f.write("%s\n" % item)

In [0]:
import json

with open('ThailandPrediction2019Future.json', 'w') as f:
    f.write("[")
    for item in row_list:
        f.write("%s,\n" % item)
    f.write("]")

In [0]:
import datetime
import pandas as pd
import reverse_geocoder as rg 
mydate = datetime.date(2019,1,1)  #year, month, day
i=0
row_list=[]

for i in range(87):
  print(mydate)
  for j in range(19):
    revGeo=dict.fromkeys(['date','lat','long'])#,'state','country','place'])
    revGeo['date']=str(mydate)
    revGeo['lat']= W[i,j,69]
    revGeo['long']=W[i,j,70]
    #print(W[i,j,69],W[i,j,70])
    coordinates = (W[i,j,69],W[i,j,70])
    print(coordinates)
    result = rg.search(coordinates)
    #print(result[0])
    #print(result[0]["lat"])
    if(result[0]['cc']=='TH'):
      #print(result[0]['cc'])
      revGeo['lat']=result[0]['lat']
      revGeo['long']=result[0]['lon']
      revGeo['state']=result[0]['admin1']
      revGeo['country']=result[0]['cc']
      revGeo['place']=result[0]['name']
    row_list.append(revGeo)
  mydate=mydate+datetime.timedelta(days=1)
  #print()
  #print()


2019-01-01
(10.601941, 100.61221)
Loading formatted geocoded file...
(11.867487, 100.567215)
(14.162363, 100.631)
(13.924614, 100.5483)
(14.337565, 100.48963)
(12.157517, 100.497284)
(13.245128, 99.955284)
(13.886083, 100.20036)
(13.6462345, 100.26531)
(13.230829, 100.17345)
(13.393473, 100.20793)
(12.648937, 100.387054)
(12.674692, 100.51584)
(12.835325, 100.570625)
(15.296286, 102.39026)
(15.998839, 102.43685)
(15.976765, 102.463196)
(16.424091, 102.84521)
(16.39053, 102.82018)
2019-01-02
(10.602114, 100.612076)
(11.867468, 100.5673)
(14.162258, 100.631004)
(13.924619, 100.54833)
(14.337431, 100.48973)
(12.157618, 100.49745)
(13.245126, 99.95517)
(13.886392, 100.20047)
(13.646517, 100.26535)
(13.230983, 100.17348)
(13.393587, 100.2079)
(12.6485815, 100.38689)
(12.67474, 100.515785)


KeyboardInterrupt: ignored

In [0]:
pd.DataFrame(row_list).to_csv("ThailandPred.csv")

In [0]:
r=pd.DataFrame(row_list)

In [0]:
W.shape

N = W.reshape(88,19*71)

In [0]:
pd.DataFrame(N).to_csv('New.csv')

In [0]:
dataframe = read_csv('glove_phase2_thailand.csv', header=0,engine='python',usecols=range(1,1351)) 
dataset = dataframe.values
dataset.shape
dataset = dataset.reshape(no_days,no_data_per_day,features)
dataset.shape
trainSet = dataset[train_size:len(dataset),:]
trainSet.shape
O = trainSet.reshape(90,19*71)

In [0]:
pd.DataFrame(O).to_csv('Old.csv')

In [0]:
dataframe = read_csv('glove_phase2_thailand.csv', header=0,engine='python',usecols=[0,70,71]) 

In [0]:
dataframe.to_csv('phase2Thailand.csv')